# Exploring Cross-Validation, Overfitting, and Underfitting

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [3]:
engagement_data = pd.read_csv('course_engagement_data.csv')

In [4]:
engagement_data

,UserID,CourseCategory,TimeSpentOnCourse,NumberOfVideosWatched,NumberOfQuizzesTaken,QuizScores,CompletionRate,DeviceType,CourseCompletion
0,5618,Health,29.979719,17,3,50.365656,20.860773,1,0
1,4326,Arts,27.802640,1,5,62.615970,65.632415,1,0
2,5849,Arts,86.820485,14,2,78.458962,63.812007,1,1
3,4992,Science,35.038427,17,10,59.198853,95.433162,0,1
4,3866,Programming,92.490647,16,0,98.428285,18.102478,0,0
...,...,...,...,...,...,...,...,...,...
8995,8757,Health,37.445225,14,4,54.469359,32.990704,1,0
8996,894,Science,48.631443,7,7,59.413257,0.254625,0,0
8997,6323,Health,38.212512,3,3,69.508297,70.188159,1,0
8998,3652,Health,70.048665,13,10,79.655182,72.975225,1,1


In [5]:
engagement_data.head()

,UserID,CourseCategory,TimeSpentOnCourse,NumberOfVideosWatched,NumberOfQuizzesTaken,QuizScores,CompletionRate,DeviceType,CourseCompletion
0,5618,Health,29.979719,17,3,50.365656,20.860773,1,0
1,4326,Arts,27.802640,1,5,62.615970,65.632415,1,0
2,5849,Arts,86.820485,14,2,78.458962,63.812007,1,1
3,4992,Science,35.038427,17,10,59.198853,95.433162,0,1
4,3866,Programming,92.490647,16,0,98.428285,18.102478,0,0


In [6]:
engagement_data.shape

(9000, 9)

In [7]:
engagement_data.describe

<bound method NDFrame.describe of       UserID CourseCategory  TimeSpentOnCourse  NumberOfVideosWatched  \
0       5618         Health          29.979719                     17   
1       4326           Arts          27.802640                      1   
2       5849           Arts          86.820485                     14   
3       4992        Science          35.038427                     17   
4       3866    Programming          92.490647                     16   
...      ...            ...                ...                    ...   
8995    8757         Health          37.445225                     14   
8996     894        Science          48.631443                      7   
8997    6323         Health          38.212512                      3   
8998    3652         Health          70.048665                     13   
8999    5595         Health          93.589781                      7   

      NumberOfQuizzesTaken  QuizScores  CompletionRate  DeviceType  \
0                  

### Check for missing values

In [11]:
engagement_data.isnull().sum()

UserID                   0
CourseCategory           0
TimeSpentOnCourse        0
NumberOfVideosWatched    0
NumberOfQuizzesTaken     0
QuizScores               0
CompletionRate           0
DeviceType               0
CourseCompletion         0
dtype: int64

TypeError: Could not convert ['...'] to numeric

In [12]:
features = engagement_data.drop(columns=['UserID', 'CourseCompletion'])
target = engagement_data['CourseCompletion']

In [13]:
features

,CourseCategory,TimeSpentOnCourse,NumberOfVideosWatched,NumberOfQuizzesTaken,QuizScores,CompletionRate,DeviceType
0,Health,29.979719,17,3,50.365656,20.860773,1
1,Arts,27.802640,1,5,62.615970,65.632415,1
2,Arts,86.820485,14,2,78.458962,63.812007,1
3,Science,35.038427,17,10,59.198853,95.433162,0
4,Programming,92.490647,16,0,98.428285,18.102478,0
...,...,...,...,...,...,...,...
8995,Health,37.445225,14,4,54.469359,32.990704,1
8996,Science,48.631443,7,7,59.413257,0.254625,0
8997,Health,38.212512,3,3,69.508297,70.188159,1
8998,Health,70.048665,13,10,79.655182,72.975225,1


In [20]:
engagement_data.columns

Index(['UserID', 'CourseCategory', 'TimeSpentOnCourse',
       'NumberOfVideosWatched', 'NumberOfQuizzesTaken', 'QuizScores',
       'CompletionRate', 'DeviceType', 'CourseCompletion'],
      dtype='object')

In [25]:
features = pd.get_dummies(features, columns=['QuizScores', 'CompletionRate'])
label_encoder = LabelEncoder()
target_encoded = label_encoder.fit_transform(target)

### Standardize the numerical features

In [27]:
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

### Split the data into training and testing sets

In [28]:
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target_encoded, test_size=0.20, random_state=42)

### Cross Valication

In [29]:
model = LogisticRegression()

In [34]:
cv_scores = cross_val_score(model, X_train, y_train, cv=5)
cv_scores

array([0.60555556, 0.60625   , 0.60625   , 0.60625   , 0.60694444])

In [36]:
cv_scores.mean()

0.60625

### Overfitting and Underfitting

In [37]:
train_accuracies = []
test_accuracies = []

### Varying the maximum depth of the tree

In [ ]:
max_depth_range = range(1, 20)
for max_depth in max_depth_range:
    tree = DecisionTreeClassifier(max_depth=max_depth, random_state=42)
    tree.fit(X_train, y_train)
    
    train_accuracies.append(tree.score(X_train, y_train))
    test_accuracies.append(tree.score(X_test, y_test))

### Plotting the training and validation accuracies

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(max_depth_range, train_accuracies, label='Training Accuracy')
plt.plot(max_depth_range, test_accuracies, label='Validation Accuracy')
plt.xlabel('Max Depth')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy vs. Max Depth')
plt.show()